# Fleet Value Analysis: Keep vs. Sell Decision Engine

## Objective
This notebook captures the end-to-end workflow for analyzing the fleet's performance. 
Our goal is to generate a **Master Decision Table** that recommends whether to **KEEP**, **SELL**, or **INSPECT** each truck based on:

1.  **Usage**: Is the truck running? (Miles, Active Days)
2.  **Revenue**: Is it earning money? (Revenue per Mile)
3.  **Cost**: Is it expensive to maintain? (Repair CPM)
4.  **Market Value**: Can we exit profitably? (Net Exit Gain)

---

In [1]:
# --- CONFIGURATION & IMPORTS ---
import pandas as pd
import numpy as np
import os
import logging

# Set Paths
DATA_DIR = r"c:\Users\ngang\OneDrive\Desktop\Projects\Data Science\fleet_analysis\data"
LOG_FILE = os.path.join(DATA_DIR, "analysis_log.txt")
OUTPUT_FILE = os.path.join(DATA_DIR, "fleet_value_report.csv")

# Setup Logging to see progress
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.INFO)

FILES = {
    "finance": "truck-finance.xlsx",
    "maintenance": "maintenancepo-truck.xlsx",
    "distance": "vehicle-distance-traveled.xlsx",
    "stub": "stub-data.xlsx",
    "market": "truck-paper.xlsx"
}

print("Configuration Loaded.")

Configuration Loaded.


## Step 1: Data Loading & Standardization
We load data from multiple disparate sources. A helper function `load_data` handles the reading (specifically targeting Sheet 2) and cleans column names.

In [2]:
def load_data(file_key, filename):
    path = os.path.join(DATA_DIR, filename)
    print(f"Loading {file_key} from {filename}...")
    try:
        # Load Sheet 2 (Index 1) as discovered in exploration
        df = pd.read_excel(path, sheet_name=1)
        df.columns = df.columns.astype(str).str.strip()
        print(f"  -> Loaded {len(df)} rows.")
        return df
    except Exception as e:
        print(f"  [ERROR] Failed to load {filename}: {e}")
        return None

# Load all datasets
finance_df = load_data("finance", FILES["finance"])
maintenance_df = load_data("maintenance", FILES["maintenance"])
distance_df = load_data("distance", FILES["distance"])
stub_df = load_data("stub", FILES["stub"])
market_df = load_data("market", FILES["market"])

Loading finance from truck-finance.xlsx...
  -> Loaded 968 rows.
Loading maintenance from maintenancepo-truck.xlsx...
  -> Loaded 1854 rows.
Loading distance from vehicle-distance-traveled.xlsx...
  -> Loaded 75858 rows.
Loading stub from stub-data.xlsx...
  -> Loaded 16108 rows.
Loading market from truck-paper.xlsx...
  -> Loaded 5366 rows.


## Step 2: Processing Operations Data
Here we ask: **"Is the truck active and earning?"**

-   **Revenue**: Aggregated from Stub entries.
-   **Distance**: Summed from daily logs (Last 10 weeks focus to see recent activity).

In [3]:
# 1. Revenue Aggregation
revenue_agg = pd.DataFrame(columns=['Unit ID', 'Total_Revenue', 'Stub_Count'])
if stub_df is not None:
    stub_df['Unit ID'] = stub_df['TRUCK'].astype(str).str.strip()
    stub_df['Revenue'] = pd.to_numeric(stub_df['TRUCK CHARGE'], errors='coerce').fillna(0)
    revenue_agg = stub_df.groupby('Unit ID').agg(
        Total_Revenue=('Revenue', 'sum'),
        Stub_Count=('Revenue', 'count')
    ).reset_index()

# 2. Distance Aggregation (Last 70 Days)
usage_agg = pd.DataFrame(columns=['Unit ID', 'Total_Miles', 'Active_Days', 'Avg_Miles_Active_Day'])
if distance_df is not None:
    distance_df['Unit ID'] = distance_df['unit_id'].astype(str).str.strip()
    distance_df['Date'] = pd.to_datetime(distance_df['date'], errors='coerce')
    distance_df['Distance'] = pd.to_numeric(distance_df['distance'], errors='coerce').fillna(0)
    
    max_date = distance_df['Date'].max()
    cutoff = max_date - pd.Timedelta(days=70)
    recent = distance_df[distance_df['Date'] > cutoff]
    
    usage_agg = recent.groupby('Unit ID').agg(
        Total_Miles=('Distance', 'sum'),
        Active_Days=('Distance', lambda x: (x > 0).sum())
    ).reset_index()
    usage_agg['Avg_Miles_Active_Day'] = usage_agg['Total_Miles'] / usage_agg['Active_Days'].replace(0, 1)

print("Operations data processed.")

Operations data processed.


## Step 3: Financials & Maintenance
Here we calculate the **Cost of Ownership**.
-   **Maintenance**: Sum of all repair orders.
-   **Finance**: Monthly payments and payoff balances.

In [4]:
# Maintenance
maint_agg = pd.DataFrame(columns=['Unit ID', 'Total_Repair_Cost', 'Repair_Count'])
if maintenance_df is not None:
    maintenance_df['Unit ID'] = maintenance_df['unit_id'].astype(str).str.strip()
    maintenance_df['Cost'] = pd.to_numeric(maintenance_df['amount'], errors='coerce').fillna(0)
    maint_agg = maintenance_df.groupby('Unit ID').agg(
        Total_Repair_Cost=('Cost', 'sum'),
        Repair_Count=('Cost', 'count')
    ).reset_index()

# Finance Base
finance_agg = finance_df[['unit_id', 'ownership_type', 'monthly_payment', 'balloon_payment', 'year', 'make', 'model']].copy()
finance_agg.columns = ['Unit ID', 'Ownership', 'Monthly_Payment', 'Payoff_Balance', 'Year', 'Make', 'Model']
for col in ['Monthly_Payment', 'Payoff_Balance']:
    finance_agg[col] = pd.to_numeric(finance_agg[col], errors='coerce').fillna(0)

print("Finance and Maintenance processed.")

Finance and Maintenance processed.


## Step 4: Market Value Estimation
We join our fleet data with scraped **Truck Paper** data to estimate what each truck is worth if sold today.
We match on `Year`, `Make`, and `Model`.

In [5]:
finance_agg['Estimated_Market_Value'] = 0.0
if market_df is not None:
    # Standardize Market Data
    market_df['Price'] = pd.to_numeric(market_df['truck_price'], errors='coerce')
    market_df['Year'] = pd.to_numeric(market_df['truck_year'], errors='coerce')
    market_df['Make_Norm'] = market_df['truck_brand'].astype(str).str.lower().str.strip()
    # Take first word of model for fuzzy matching (e.g. 'Cascadia' from 'Cascadia 125')
    market_df['Model_Norm'] = market_df['truck_model'].astype(str).str.lower().str.strip().str.split(' ').str[0]
    
    # Calculate Median Price per Segment
    market_stats = market_df.groupby(['Year', 'Make_Norm', 'Model_Norm'])['Price'].median().reset_index()
    
    # Match Fleet Data
    finance_agg['Year_Num'] = pd.to_numeric(finance_agg['Year'], errors='coerce')
    finance_agg['Make_Norm'] = finance_agg['Make'].astype(str).str.lower().str.strip()
    finance_agg['Model_Norm'] = finance_agg['Model'].astype(str).str.lower().str.strip().str.split(' ').str[0]
    
    temp = finance_agg.merge(market_stats, left_on=['Year_Num', 'Make_Norm', 'Model_Norm'], right_on=['Year', 'Make_Norm', 'Model_Norm'], how='left')
    finance_agg['Estimated_Market_Value'] = temp['Price'].fillna(0)
    
    # Fallback for unmatched
    avg_price = market_df['Price'].median()
    finance_agg['Estimated_Market_Value'] = finance_agg['Estimated_Market_Value'].replace(0, avg_price)
    print("Market values estimated.")

Market values estimated.


## Step 5: The Master Table
We merge details into a single view per truck.

In [6]:
master = finance_agg.merge(usage_agg, on='Unit ID', how='left')
master = master.merge(revenue_agg, on='Unit ID', how='left')
master = master.merge(maint_agg, on='Unit ID', how='left')

# Fill NaNs with 0 for calculation safety
fill_cols = ['Total_Miles', 'Active_Days', 'Total_Revenue', 'Stub_Count', 'Total_Repair_Cost', 'Repair_Count', 'Avg_Miles_Active_Day', 'Estimated_Market_Value']
for col in fill_cols:
    if col in master.columns:
         master[col] = master[col].fillna(0)
    else:
         master[col] = 0

print(f"Master table compiled: {len(master)} units.")

Master table compiled: 968 units.


## Step 6: Decision Logic & Scoring
We calculate derived metrics and apply business rules.

### Key Metrics
- **Repair CPM**: Cost per Mile for repairs. High CPM (> $0.80) indicates a money pit.
- **Revenue Per Mile**: Efficiency of earnings.
- **Net Exit Gain**: Market Value - Payoff Balance. Positive means we get cash if we sell.

### Rules
- **KEEP (Profitable)**: Good revenue, currently active.
- **KEEP (Underwater)**: We owe more than it's worth, so we hold.
- **SELL (High Maint)**: Repair costs are destroying margins.
- **SELL (Low Use)**: Asset is sitting idle but has value.

In [7]:
# 1. Calculate Metrics
master['Repair_CPM'] = np.where(master['Total_Miles'] > 0, 
                                master['Total_Repair_Cost'] / master['Total_Miles'], 
                                0)
master['Revenue_Per_Mile'] = np.where(master['Total_Miles'] > 0,
                                      master['Total_Revenue'] / master['Total_Miles'],
                                      0)
master['Net_Exit_Gain'] = master['Estimated_Market_Value'] - master['Payoff_Balance']

# 2. Decision Function
def recommend(row):
    # Condition 1: Costly but not running
    if row['Total_Miles'] == 0 and row['Total_Repair_Cost'] > 0: return 'INSPECT (Cost No Miles)'
    
    # Condition 2: Sell Opportunities
    if row['Net_Exit_Gain'] > 5000:
        if row['Total_Miles'] < 2000: return 'SELL (Low Use, High Value)'
        if row['Repair_CPM'] > 0.80: return 'SELL (High Maint)'
        
    # Condition 3: Keep Reasons
    if row['Total_Revenue'] > row['Monthly_Payment'] * 2: return 'KEEP (Profitable)'
    if row['Net_Exit_Gain'] < -5000: return 'KEEP (Underwater)'
    
    return 'INSPECT (Review)'

master['Recommendation'] = master.apply(recommend, axis=1)

# 3. Projection (10 Weeks)
master['Projected_Net_Value_10Wks'] = (
    (master['Total_Miles'] * master['Revenue_Per_Mile']) - 
    (master['Total_Repair_Cost']) - 
    (master['Monthly_Payment'] * 2.5)
)

print("Decision logic applied.")

Decision logic applied.


## Conclusion & Export
The analysis is complete. We export the table for review.

In [8]:
cols = [
    'Unit ID', 'Recommendation', 'Make', 'Model', 'Year',
    'Total_Miles', 'Total_Revenue', 'Total_Repair_Cost', 
    'Repair_CPM', 'Revenue_Per_Mile', 'Net_Exit_Gain', 'Projected_Net_Value_10Wks',
    'Active_Days', 'Stub_Count',
    'Ownership', 'Monthly_Payment', 'Payoff_Balance', 'Estimated_Market_Value'
]

final_df = master[cols]
print(final_df['Recommendation'].value_counts())
final_df.head(10)

Recommendation
SELL (Low Use, High Value)    629
KEEP (Profitable)             191
INSPECT (Review)              120
INSPECT (Cost No Miles)        18
SELL (High Maint)              10
Name: count, dtype: int64


,Unit ID,Recommendation,Make,Model,Year,Total_Miles,Total_Revenue,Total_Repair_Cost,Repair_CPM,Revenue_Per_Mile,Net_Exit_Gain,Projected_Net_Value_10Wks,Active_Days,Stub_Count,Ownership,Monthly_Payment,Payoff_Balance,Estimated_Market_Value
0,Truck 0003,"SELL (Low Use, High Value)",INTERNATIONAL,TT,2020,0.0,0.0,0.0,0.0,0.0,57900.0,0.0,0.0,0.0,Owner operator owned,0.0,0.0,57900.0
1,Truck 0008,INSPECT (Review),Freightliner,TT,2016,5437.0,0.0,0.0,0.0,0.0,57900.0,0.0,13.0,38.0,Owner operator owned,0.0,0.0,57900.0
2,Truck 0013,INSPECT (Review),Western Star,TT,2007,24541.0,0.0,0.0,0.0,0.0,57900.0,0.0,57.0,61.0,Owner Operator Owned,0.0,0.0,57900.0
3,Truck 0002,INSPECT (Review),International,TT,2019,29897.0,0.0,0.0,0.0,0.0,57900.0,0.0,60.0,12.0,Owner operator owned,0.0,0.0,57900.0
4,Truck 0022,"SELL (Low Use, High Value)",kenworth,TT,2006,0.0,0.0,0.0,0.0,0.0,57900.0,0.0,0.0,0.0,NaN,0.0,0.0,57900.0
5,Truck 0007,INSPECT (Review),Kenworth,TT,2021,30320.0,0.0,0.0,0.0,0.0,57900.0,0.0,56.0,42.0,Owner Operator Owned,0.0,0.0,57900.0
6,Truck 0012,INSPECT (Review),VOLVO,TT,2018,23972.0,0.0,0.0,0.0,0.0,57900.0,0.0,57.0,20.0,Owner operator owned,0.0,0.0,57900.0
7,Truck 0035,"SELL (Low Use, High Value)",International,TT,2006,1155.0,0.0,0.0,0.0,0.0,57900.0,0.0,7.0,5.0,Owner Operator Owned,0.0,0.0,57900.0
8,Truck 0040,"SELL (Low Use, High Value)",freightliner,TT,2016,554.0,0.0,0.0,0.0,0.0,57900.0,0.0,1.0,3.0,Owner operator owned,0.0,0.0,57900.0
9,Truck 0045,"SELL (Low Use, High Value)",freightliner,TT,2018,0.0,0.0,0.0,0.0,0.0,57900.0,0.0,0.0,0.0,Owner operator owned,0.0,0.0,57900.0
